# Финальный проект

In [2]:
import requests
from pprint import pprint
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)
import csv

In [3]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [4]:
session = requests.session()

Притворяемся браузером

In [5]:
ua = UserAgent(verify_ssl=False)

## Код для сбора текстов для корпуса

In [6]:
url = f'https://rustih.ru/stixi-o-prirode/'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')

corpora = {}

for element in soup.find_all('div', {'class': 'entry-title'}):
    title = element.text
    link = element.find('a').attrs['href']
    if link != 'https://rustih.ru/fedor-tyutchev-ne-to-chto-mnite-vy-priroda/': # почему-то спотыкается, а еще там _______ как разделитель используется неожиданно, поэтому это проигнорируем
        req_poem = session.get(link, headers={'User-Agent': ua.random})
        poem = req_poem.text
        soup_poem = BeautifulSoup(poem, 'html.parser')
        text = soup_poem.find('div', {'class': 'entry-content poem-text'}).text.replace('I', '')    # У некоторых стихотворений указываются номера четверостиший римскими цифрами, поэтому нам нужно избавиться от букв, которые используются при записи римским цифр
        text = text.replace('X', '')
        text = text.replace('V', '')
        text = sent_tokenize(text.lstrip())
        cleared_text = text.copy()
        for sent in text:
            if len(sent) < 3:
                cleared_text.remove(sent)
            elif '______' in sent:  #Эта штука используется как разделитель между стихотворением и его анализом, анализ нам не слишком нужен, поэтому можно убирать
                cleared_text = cleared_text[:cleared_text.index(sent)]
        corpora[title] = cleared_text
        if len(corpora) == 100:
            break

## Код для сбора всех текстов в корпус

In [7]:
with open('my_corpus.csv', 'a', encoding='utf8')as f:
    fieldnames = ['id_sent', 'id_word', 'word', 'POS', 'lemma', 'sentence']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    i = 0

    for k,v in corpora.items():
        for st in v:
            doc = Doc(st)
            sent = str(st) + ' [' + str(k) + ']\n'
            doc.segment(segmenter)
            doc.tag_morph(morph_tagger)
            for token in doc.tokens:
                token.lemmatize(morph_vocab)
            j = 0
            for el in doc.tokens:
                writer.writerow({'id_sent': i, 'id_word': j, 'word': el.text.lower(),
                'POS': el.pos, 'lemma': el.lemma, 'sentence': sent})
                j += 1
            i += 1

## Код для поиска по корпусу

In [8]:
tags = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']

Функция, которая осуществляем поиск по корпусу

In [39]:
def searching(query):
    query = query.split()
    result = []
    id = ''
    word = ''
    s = ''

    with open('my_corpus.csv', encoding='utf8') as f:
        reader = csv.DictReader(f)

        if len(query) == 1: # Если запрос состоит из одного слова
            if query[0] not in tags and query[0][0] != '"':
                q = query[0]
                if '+' in query[0]:
                    q = query[0].split('+')[0]
                doc = Doc(q)
                doc.segment(segmenter)
                doc.tag_morph(morph_tagger)
                for token in doc.tokens:
                    token.lemmatize(morph_vocab)

            for row in reader:
                if id:
                    if row['id_sent'] == id:    # Если это предложение уже записано в выдачу, то мы игнорируем все оставшиеся слова из него, потому что нет смысла несколько раз выводить одно и то же предложение
                        continue
                if query[0] in tags:
                    if row['POS'] == query[0]:
                        result.append(row['sentence'])
                        id = row['id_sent']
                elif query[0][0] == '"':
                    if row['word'] == query[0][1:len(query[0])-1]:
                        result.append(row['sentence'])
                        id = row['id_sent']
                elif '+' in query[0]:
                    w, p = query[0].split('+')
                    if doc.tokens[0].lemma == row['lemma'] and p == row['POS']:
                        result.append(row['sentence'])
                        id = row['id_sent']
                else:
                    if row['lemma'] == doc.tokens[0].lemma or row['lemma'] == query[0]:
                        result.append(row['sentence'])
                        id = row['id_sent']

        elif len(query) == 2:   # Если запрос состоит из двух слов
            if query[0] not in tags and query[0][0] != '"':
                q1 = query[0]
                if '+' in query[0]:
                    q1 = query[0].split('+')[0]
                doc1 = Doc(q1)
                doc1.segment(segmenter)
                doc1.tag_morph(morph_tagger)
                for token in doc1.tokens:
                    token.lemmatize(morph_vocab)

            if query[1] not in tags and query[1][0] != '"':
                q2 = query[1]
                if '+' in query[1]:
                    q2 = query[1].split('+')[0]
                doc2 = Doc(q2)
                doc2.segment(segmenter)
                doc2.tag_morph(morph_tagger)
                for token in doc2.tokens:
                    token.lemmatize(morph_vocab)
                    
            for row in reader:
                if id:
                    if id == row['id_sent']:
                        continue

                if query[0] in tags:
                    if row['POS'] == query[0]:
                        word = row['id_word']
                        s = row['id_sent']
                elif query[0][0] == '"':
                    if row['word'] == query[0][1:len(query[0])-1]:
                        word = row['id_word']
                        s = row['id_sent']
                elif '+' in query[0]:
                        w, p = query[0].split('+')
                        if doc1.tokens[0].lemma == row['lemma'] and p == row['POS']:
                            word = row['id_word']
                            s = row['id_sent']
                else:
                        if row['lemma'] == doc1.tokens[0].lemma or row['lemma'] == query[0]:
                            word = row['id_word']
                            s = row['id_sent']

                if s != row['id_sent']:
                    word = ''

                if word:
                    if query[1] in tags:
                        if row['POS'] == query[1] and int(word) + 1 == int(row['id_word']):
                            result.append(row['sentence'])
                            id = row['id_sent']
                    elif query[1][0] == '"':
                        if row['word'] == query[1][1:len(query[1])-1] and int(word) + 1 == int(row['id_word']):
                            result.append(row['sentence'])
                            id = row['id_sent']
                    elif '+' in query[1]:
                        w, p = query[1].split('+')
                        if doc2.tokens[0].lemma == row['lemma'] and p == row['POS'] and int(word) + 1 == int(row['id_word']):
                            result.append(row['sentence'])
                            id = row['id_sent']
                    else:
                        if (row['lemma'] == doc2.tokens[0].lemma or row['lemma'] == query[0]) and int(word) + 1 == int(row['id_word']):
                            result.append(row['sentence'])
                            id = row['id_sent']

        elif len(query) == 3:   # Если запрос состоит из трех слов
            if query[0] not in tags and query[0][0] != '"':
                q1 = query[0]
                if '+' in query[0]:
                    q1 = query[0].split('+')[0]
                doc1 = Doc(q1)
                doc1.segment(segmenter)
                doc1.tag_morph(morph_tagger)
                for token in doc1.tokens:
                    token.lemmatize(morph_vocab)
            if query[1] not in tags and query[1][0] != '"':
                q2 = query[1]
                if '+' in query[1]:
                    q2 = query[1].split('+')[0]
                doc2 = Doc(q2)
                doc2.segment(segmenter)
                doc2.tag_morph(morph_tagger)
                for token in doc2.tokens:
                    token.lemmatize(morph_vocab)
            if query[2] not in tags and query[2][0] != '"':
                q3 = query[2]
                if '+' in query[2]:
                    q3 = query[2].split('+')[0]
                doc3 = Doc(q3)
                doc3.segment(segmenter)
                doc3.tag_morph(morph_tagger)
                for token in doc3.tokens:
                    token.lemmatize(morph_vocab)
                    
            for row in reader:
                if id:
                    if id == row['id_sent']:
                        continue

                if query[0] in tags:
                    if row['POS'] == query[0]:
                        word = row['id_word']
                        s = row['id_sent']

                elif query[0][0] == '"':
                    if row['word'] == query[0][1:len(query[0])-1]:
                        word = row['id_word']
                        s = row['id_sent']

                elif '+' in query[0]:
                        w, p = query[0].split('+')
                        if doc1.tokens[0].lemma == row['lemma'] and p == row['POS']:
                            word = row['id_word']
                            s = row['id_sent']

                else:
                        if row['lemma'] == doc1.tokens[0].lemma or row['lemma'] == query[0]:
                            word = row['id_word']
                            s = row['id_sent']

                if s != row['id_sent']:
                    word = ''

                if word:
                    if query[1] in tags:
                        if row['POS'] == query[1] and int(word) + 1 == int(row['id_word']):
                            word = row['id_word']
                    elif query[1][0] == '"':
                        if row['word'] == query[1][1:len(query[1])-1] and int(word) + 1 == int(row['id_word']):
                            word = row['id_word']
                    elif '+' in query[1]:
                            w, p = query[1].split('+')
                            if doc2.tokens[0].lemma == row['lemma'] and p == row['POS'] and int(word) + 1 == int(row['id_word']):
                                word = row['id_word']
                    else:
                            if (row['lemma'] == doc2.tokens[0].lemma or row['lemma'] == query[1]) and int(word) + 1 == int(row['id_word']):
                                word = row['id_word']

                    if query[2] in tags:
                        if row['POS'] == query[2] and int(word) + 1 == int(row['id_word']):
                            result.append(row['sentence'])
                            id = row['id_sent']
                    elif query[2][0] == '"':
                        if row['word'] == query[2][1:len(query[2])-1] and int(word) + 1 == int(row['id_word']):
                            result.append(row['sentence'])
                            id = row['id_sent']
                    elif '+' in query[2]:
                            w, p = query[2].split('+')
                            if doc3.tokens[0].lemma == row['lemma'] and p == row['POS'] and int(word) + 1 == int(row['id_word']):
                                result.append(row['sentence'])
                                id = row['id_sent']
                    else:
                            if (row['lemma'] == doc3.tokens[0].lemma or row['lemma'] == query[2]) and int(word) + 1 == int(row['id_word']):
                                result.append(row['sentence'])
                                id = row['id_sent']
    return result

Здесь можно сделать запрос и посмотреть на выдачу

In [46]:
searched = input()
searching(searched)

['На весь его недолгий роздых\nМы целый дом ему сдаем. [\xa0Борис Пастернак — Июль]\n',
 'Для Есенина, уже познакомившегося со столичной жизнью и успевшего на нее насмотреться, береза была также символом родного дома. [\xa0Сергей Есенин — Белая береза под моим окном]\n',
 'Ветвь и пальма символизируют любящих людей, тоскующих в разлуке, а так же разобщенные части одной души, некую оторванность от родного дома (от самого себя). [\xa0Михаил Лермонтов — Ветка Палестины]\n',
 'Теперь становится ясно, что защитить от жизненных невзгод могут только личные силы героя, настрой на позитивное, а не стены родного дома. [Александр Пушкин — Зимний вечер (Буря мглою небо кроет)]\n']